In [1]:
# Importing the required libraries and dependencies
import numpy as np
import pandas as pd
import random
from pathlib import Path
from sqlalchemy import create_engine, inspect, MetaData, Table, Column, Integer, String, types

In [2]:
database_connection_string = 'sqlite:///debtfreedb.db'

engine = create_engine(database_connection_string)

In [3]:
# load the mcc code csv
csv_path = Path('./resources/mcc_codes.csv')
mcc_df = pd.read_csv(csv_path)
mcc_df.head()

,mcc,edited_description,combined_description,usda_description,irs_description,irs_reportable
0,742,Veterinary Services,Veterinary Services,Veterinary Services,Veterinary Services,Yes
1,763,Agricultural Co-operatives,Agricultural Co-operatives,Agricultural Co-operatives,Agricultural Cooperative,Yes
2,780,"Horticultural Services, Landscaping Services","Horticultural Services, Landscaping Services",Horticultural Services,Landscaping Services,Yes
3,1520,General Contractors-Residential and Commercial,General Contractors-Residential and Commercial,General Contractors-Residential and Commercial,General Contractors,Yes
4,1711,Air Conditioning Contractors – Sales and Insta...,Air Conditioning Contractors – Sales and Insta...,Air Conditioning Contractors – Sales and Insta...,"Heating, Plumbing, A/C",Yes


In [4]:
# Change datatype
mcc_df['mcc'] = mcc_df['mcc'].astype(str)
# Change Value
mcc_df['mcc'] = mcc_df['mcc'].str.zfill(4)

# create mcc_type table
mcc_df[['mcc','edited_description' ]].to_sql(name="mcc_type", con=engine, if_exists='replace', index=False,
            dtype={'mcc': types.VARCHAR(length=4), 
                   'edited_description':  types.VARCHAR(length=150)})


981

In [5]:
# load the credit card info csv
csv_path = Path('./ccinfo/ccinfo_reduced.csv')
cc_info_df = pd.read_csv(csv_path)
cc_info_df.head()


,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2016,1,3,10:48,$66.48,Chip Transaction,-3345936507911876459,La Verne,CA,91750.0,7538,NaN,No
1,0,0,2016,1,4,06:43,$40.02,Chip Transaction,-34551508091458520,La Verne,CA,91750.0,5912,NaN,No
2,0,0,2016,1,7,09:30,$54.11,Chip Transaction,4055257078481058705,La Verne,CA,91750.0,7538,NaN,No
3,0,0,2016,1,7,16:03,$89.48,Chip Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2016,1,10,06:38,$29.15,Chip Transaction,-5475680618560174533,Monterey Park,CA,91755.0,5942,NaN,No


In [6]:
# Change data type
cc_info_df['MCC'] = cc_info_df['MCC'].astype(str)
cc_info_df['Zip'] = cc_info_df['Zip'].astype(str)
cc_info_df['Amount'] = cc_info_df['Amount'].astype(str)
cc_info_df['Month'] = cc_info_df['Month'].astype(str)
cc_info_df['Day'] = cc_info_df['Day'].astype(str)

# Change/reformat the value
cc_info_df['MCC'] = cc_info_df['MCC'].str.zfill(4)
cc_info_df['Month'] = cc_info_df['Month'].str.zfill(2)
cc_info_df['Day'] = cc_info_df['Day'].str.zfill(2)
cc_info_df['zip'] = cc_info_df['Zip'].str[:5]
cc_info_df['amount'] = cc_info_df['Amount'].str[1:]
cc_info_df['user_name'] = 'USER' + cc_info_df['User'].astype(str)
cc_info_df['merchant_id'] = cc_info_df['Merchant Name']

cc_info_df['txn_date'] = cc_info_df['Year'].astype(str) + '/' + cc_info_df['Month'] + '/' + cc_info_df['Day'] + ' ' + cc_info_df['Time'] 
cc_info_df['transaction_date'] = pd.to_datetime(cc_info_df['txn_date'])

# Change column names
cc_info_df.rename(columns={'User':'user_id',
                           'MCC':'mcc',
                           'Card':'account_id',
                           'Merchant Name':'merchant_name',
                           'Merchant City':'merchant_city',
                           'Merchant State':'merchant_state'
                          }, inplace=True)


cc_info_df.head()

,user_id,account_id,Year,Month,Day,Time,Amount,Use Chip,merchant_name,merchant_city,...,Zip,mcc,Errors?,Is Fraud?,zip,amount,user_name,merchant_id,txn_date,transaction_date
0,0,0,2016,01,03,10:48,$66.48,Chip Transaction,-3345936507911876459,La Verne,...,91750.0,7538,NaN,No,91750,66.48,USER0,-3345936507911876459,2016/01/03 10:48,2016-01-03 10:48:00
1,0,0,2016,01,04,06:43,$40.02,Chip Transaction,-34551508091458520,La Verne,...,91750.0,5912,NaN,No,91750,40.02,USER0,-34551508091458520,2016/01/04 06:43,2016-01-04 06:43:00
2,0,0,2016,01,07,09:30,$54.11,Chip Transaction,4055257078481058705,La Verne,...,91750.0,7538,NaN,No,91750,54.11,USER0,4055257078481058705,2016/01/07 09:30,2016-01-07 09:30:00
3,0,0,2016,01,07,16:03,$89.48,Chip Transaction,3414527459579106770,Monterey Park,...,91754.0,5651,NaN,No,91754,89.48,USER0,3414527459579106770,2016/01/07 16:03,2016-01-07 16:03:00
4,0,0,2016,01,10,06:38,$29.15,Chip Transaction,-5475680618560174533,Monterey Park,...,91755.0,5942,NaN,No,91755,29.15,USER0,-5475680618560174533,2016/01/10 06:38,2016-01-10 06:38:00


In [7]:
# create transaction table
cc_info_df[['transaction_date','account_id', 'amount', 'mcc', 'merchant_id']].to_sql(name="transaction_data", con=engine, if_exists='replace', index=False,
            dtype={'transaction_date': types.DateTime(timezone=False), 
                   'account_id': types.BigInteger(),
                   'amount': types.Numeric(8,2),
                   'mcc': types.VARCHAR(length=4),
                   'merchant_id': types.BigInteger()
                  })



185169

In [8]:
# create dataframe for account holder
account_holder_df = cc_info_df[['user_id', 'user_name']]
account_holder_df.drop_duplicates(inplace=True)
account_holder_df.head()

/var/folders/0r/21lfq05d02q2rck_592dq44c0000gn/T/ipykernel_46422/2271593581.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  account_holder_df.drop_duplicates(inplace=True)


,user_id,user_name
0,0,USER0
4835,1,USER1
7180,2,USER2
18423,3,USER3
22056,4,USER4


In [9]:
# create account holder table
account_holder_df[['user_id','user_name']].to_sql(name="account_holder", con=engine, if_exists='replace', index=False,
            dtype={'user_id': types.BigInteger(), 
                   'user_name': types.VARCHAR(length=50)
                  })

49

In [10]:
# create dataframe for accounts
accounts_df = cc_info_df[['user_id', 'account_id']]
accounts_df.drop_duplicates(inplace=True)
accounts_df['credit_limit'] = accounts_df['account_id'] * random.randint(1000,25000) + random.randint(1000,25000)
accounts_df.head()

/var/folders/0r/21lfq05d02q2rck_592dq44c0000gn/T/ipykernel_46422/4101176919.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accounts_df.drop_duplicates(inplace=True)
/var/folders/0r/21lfq05d02q2rck_592dq44c0000gn/T/ipykernel_46422/4101176919.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accounts_df['credit_limit'] = accounts_df['account_id'] * random.randint(1000,25000) + random.randint(1000,25000)


,user_id,account_id,credit_limit
0,0,0,1446
875,0,1,23916
1722,0,2,46386
2538,0,3,68856
4835,1,1,23916


In [11]:
# create accounts table
accounts_df[['user_id','account_id','credit_limit']].to_sql(name="account", con=engine, if_exists='replace', index=False,
            dtype={'user_id': types.BigInteger(), 
                   'account_id': types.BigInteger(),
                   'credit_limit': types.Numeric(10,2)
                  })

128

In [12]:
merchants_df = cc_info_df[['merchant_id', 'merchant_name', 'merchant_city', 'merchant_state', 'zip']]

merchants_df.drop_duplicates(inplace=True)
merchants_df.head()


/var/folders/0r/21lfq05d02q2rck_592dq44c0000gn/T/ipykernel_46422/205745408.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merchants_df.drop_duplicates(inplace=True)


,merchant_id,merchant_name,merchant_city,merchant_state,zip
0,-3345936507911876459,-3345936507911876459,La Verne,CA,91750
1,-34551508091458520,-34551508091458520,La Verne,CA,91750
2,4055257078481058705,4055257078481058705,La Verne,CA,91750
3,3414527459579106770,3414527459579106770,Monterey Park,CA,91754
4,-5475680618560174533,-5475680618560174533,Monterey Park,CA,91755


In [13]:
# create accounts table
merchants_df[['merchant_id','merchant_name','merchant_city','merchant_state','zip']].to_sql(name="merchant", con=engine, if_exists='replace', index=False,
            dtype={'user_id': types.BigInteger(), 
                   'merchant_name': types.VARCHAR(length=50),
                   'merchant_city': types.VARCHAR(length=50),
                   'merchant_state': types.VARCHAR(length=2),
                   'zip': types.VARCHAR(length=5),
                  })

8518